# Data Engineering Capstone Project


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Do all imports and installs here
import sys, os
import logging
import pandas as pd
from pandas_profiling import ProfileReport
from pathlib import Path
from typing import Iterable
from IPython import display as ICD

In [3]:
src_path: str = "../src"
sys.path.append(src_path)
logging.getLogger().setLevel(logging.INFO)

In [4]:
from utils.io import process_config
from utils.aws import create_s3_bucket
from utils.spark import create_spark_session
from data.tables import (
    ON_LOAD_TABLES_SCHEMA,
    ON_LOAD_TABLES_FILES,
    STAR_EXTRACT_TABLES_ARGS,
)

In [5]:
user_config, dl_config = (
    process_config(Path(os.getcwd()).parent.joinpath("_user.cfg")),
    process_config(Path(os.getcwd()).parent.joinpath("dl.cfg")),
)
spark = create_spark_session(user_config, dl_config)
s3_bucket_prefix = dl_config.get("S3", "BUCKET_NAME")

22/12/19 08:48:12 WARN Utils: Your hostname, uzi resolves to a loopback address: 127.0.1.1; using 192.168.1.181 instead (on interface wlp114s0)
22/12/19 08:48:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/uziel/miniconda3/envs/de_capstone/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/uziel/.ivy2/cache
The jars for the packages stored in: /home/uziel/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-52e8cec5-1a22-485a-b461-73693e8ce2a0;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 178ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-----------------------------------

22/12/19 08:48:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


---

## 1. Preview raw data


In [6]:
for table_name, table_schema in ON_LOAD_TABLES_SCHEMA.items():
    table_paths = ON_LOAD_TABLES_FILES[table_name]
    table_df = spark.read.csv(
        (
            str(table_paths)
            if not isinstance(table_paths, Iterable)
            else [str(p) for p in table_paths]
        ),
        schema=ON_LOAD_TABLES_SCHEMA[table_name],
        header=True,
    )

    n_elem = table_df.count()
    table_df_preview = spark.createDataFrame(
        table_df.take(5),
        schema=ON_LOAD_TABLES_SCHEMA[table_name],
    ).toPandas()

    print(f"First 5 rows of {table_name}:")
    print(f"Columns: {table_df.columns}.")
    ICD.display(table_df_preview)
    print(f"The full table contains a total of {n_elem} records\n\n")

22/12/19 08:49:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
First 5 rows of i94_immigration:
Columns: ['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline', 'admnum', 'fltno', 'visatype'].


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,1.0,2016.0,7.0,254.0,276.0,LOS,20636.0,1.0,CA,20640.0,...,None,M,1978.0,9282016.0,M,NaN,OZ,6.309290e+10,00202,WT
1,2.0,2016.0,7.0,140.0,140.0,NYC,20636.0,1.0,NY,20657.0,...,None,M,1971.0,9282016.0,F,NaN,DL,6.309290e+10,09858,WT
2,3.0,2016.0,7.0,135.0,135.0,ORL,20636.0,1.0,FL,20657.0,...,None,M,2006.0,9282016.0,M,NaN,VS,6.309290e+10,00071,WT
3,4.0,2016.0,7.0,124.0,124.0,TAM,20636.0,1.0,FL,20645.0,...,None,M,1999.0,9282016.0,M,NaN,LH,6.309290e+10,00482,WT
4,5.0,2016.0,7.0,130.0,130.0,LOS,20636.0,1.0,CA,20662.0,...,None,M,2015.0,9282016.0,M,NaN,SU,6.309290e+10,00106,WT


The full table contains a total of 40790529 records


First 5 rows of us_demographics:
Columns: ['City', 'State', 'Median Age', 'Male Population', 'Female Population', 'Total Population', 'Number of Veterans', 'Foreign-born', 'Average Household Size', 'State Code', 'Race', 'Count'].


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.799999,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.000000,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.500000,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.500000,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.599998,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


The full table contains a total of 2891 records


First 5 rows of airport_codes:
Columns: ['ident', 'type', 'name', 'elevation_ft', 'continent', 'iso_country', 'iso_region', 'municipality', 'gps_code', 'iata_code', 'local_code', 'coordinates'].


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,None,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,None,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,None,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,None,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,None,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


The full table contains a total of 55075 records




First 5 rows of world_temperature:
Columns: ['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City', 'Country', 'Latitude', 'Longitude'].


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068000,1.737,Århus,Denmark,57.05N,10.33E
1,1744-04-01,5.788000,3.624,Århus,Denmark,57.05N,10.33E
2,1744-05-01,10.644000,1.283,Århus,Denmark,57.05N,10.33E
3,1744-06-01,14.051000,1.347,Århus,Denmark,57.05N,10.33E
4,1744-07-01,16.082001,1.396,Århus,Denmark,57.05N,10.33E


The full table contains a total of 8235082 records




---

## 2. Run ETL pipeline to extract STAR dimensional tables


Create S3 bucket to store all results


In [7]:
assert create_s3_bucket(user_config, dl_config), "Error creating S3 bucket."

INFO:root:Bucket cupm-de-capstone already exists.
INFO:root:Available buckets: [s3.Bucket(name='cupm-de-capstone')]


#### Run Airflow DAG (`capstone_etl`) now.


---

## 3. Run analytics queries on dimensional tables


In [8]:
profiling_path = Path("../data").joinpath("profiling_reports")
profiling_path.mkdir(exist_ok=True)

In [9]:
star_tables = {
    table_name: spark.read.parquet(table_args["op_kwargs"]["s3_save_path"])
    for table_name, table_args in STAR_EXTRACT_TABLES_ARGS.items()
}

22/12/19 08:49:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/12/19 08:49:13 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
22/12/19 08:49:14 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
22/12/19 08:49:14 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.


### 3.1. Data profiling of dimensional tables

WARNING: Avoid for tables with numbers of rows in the order of dozens of millions, according to memory availability.


In [10]:
for table_name, table_df in star_tables.items():
    if table_name == "fact_immigration":
        continue
    star_table = table_df.toPandas()
    ProfileReport(star_table).to_file(profiling_path.joinpath(f"{table_name}.html"))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/uziel/miniconda3/envs/de_capstone/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/uziel/miniconda3/envs/de_capstone/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### 3.2. Example queries using combinations of dimensional tables


#### Do immigrants prefer destinations with higher or lower population?


In [11]:
df = (
    star_tables["fact_immigration"]
    .groupBy("city_id")
    .count()
    .join(
        star_tables["fact_us_demogr"],
        (
            star_tables["fact_immigration"]["city_id"]
            == star_tables["fact_us_demogr"]["city_id"]
        ),
    )
    .dropna(subset=["total_population"])
    .select(["count", "total_population"])
)

In [12]:
df.stat.corr("count", "total_population")

0.7699218219771018

There is a high positive correlation between the population size of a city and the number of immigrants it attracts.


#### Do immigrants prefer destinations with higher or lower temperature?


In [13]:
df = (
    star_tables["fact_immigration"]
    .groupBy("city_id")
    .count()
    .join(
        star_tables["fact_temps"],
        (
            star_tables["fact_immigration"]["city_id"]
            == star_tables["fact_temps"]["city_id"]
        ),
    )
    .dropna(subset=["avg_temperature"])
    .select(["count", "avg_temperature"])
)

In [14]:
df.stat.corr("count", "avg_temperature")

-0.044231400827696755

There is no correlation between number of immigrants and average temperature of a city.


#### Do immigrants prefer destinations with more or less airports?


In [15]:
df = (
    star_tables["fact_immigration"]
    .groupBy("city_id")
    .count()
    .join(
        (
            star_tables["dim_airports"]
            .groupBy("city_id")
            .count()
            .withColumnRenamed("count", "airports_count")
        ),
        (
            star_tables["fact_immigration"]["city_id"]
            == star_tables["dim_airports"]["city_id"]
        ),
    )
    .select(["count", "airports_count"])
)

In [16]:
df.stat.corr("count", "airports_count")

0.37289342731132336

There is a low, positive correlation between number of immigrants and number of airports in the receiving city.

---

### Step 5: Complete Project Write Up

- Clearly state the rationale for the choice of tools and technologies for the project.
- Propose how often the data should be updated and why.
- Write a description of how you would approach the problem differently under the following scenarios:
  - The data was increased by 100x.
  - The data populates a dashboard that must be updated on a daily basis by 7am every day.
  - The database needed to be accessed by 100+ people.
